---
title: "Tutoriel de différentiation automatique"
from: markdown+emoji
lang: fr
format: html
execute:
    cache: true
toc: true
author:
  - name: Mahendra Mariadassou
    orcid: 0000-0003-2986-354X
    email: mahendra.mariadassou@inrae.fr
    affiliations:
      - name: INRAE - MaIAGE
        adress: Domaine de Vilvert
        city: Jouy-en-Josas
        state: France
  - name: Hugo Gangloff
  - name: Arthur Leroy
  - name: Lucia Clarotto
date: "2025-08-18"
date-modified: today
date-format: "[Last Updated on] MMMM, YYYY"
---


::: callout-note
L'obectif de ce tutoriel est de montrer comment utiliser JAX et PyTorch pour calculer le JVP et le VJP d'une fonction simple:
$$
f: (x, a, b) \in \mathbb{R}^p \times \mathbb{R}^p \times \mathbb{R} \mapsto \tanh(a^\top x + b) \in \mathbb{R}
$$
On considère les dérivées par rapport à l'entrée $x$ et aux paramètres $a, b$. On rappelle que $tanh'(x) = 1 - \tanh^2(x)$ et que 

$$
\frac{\partial d}{\partial x} = a.(1 - \tanh^2(a^\top x + b)) \qquad \frac{\partial d}{\partial a} = x.(1 - \tanh^2(a^\top x + b)) \qquad \frac{\partial d}{\partial b} = (1 - \tanh^2(a^\top x + b))
$$
:::

# Théorie [XX]

# Exemple en Torch [MM, LC]


In [ ]:
import torch
import numpy as np

On definit notre fonction $f$ en torch. 


In [ ]:
def f_torch(x, a, b):
    return torch.tanh(torch.dot(x, a) + b)

On définit des valeurs pour lesquelles on sait calculer facilement le gradient. 


In [ ]:
x = torch.tensor([2., 3.], requires_grad = True)
a = torch.ones(2, requires_grad = True)
b = torch.tensor(-2., requires_grad = True)

Et on calcule les dérivées partielles ($\partial f / \partial x = `x.grad`).


In [ ]:
## Définit y par rapport à x
y = f_torch(x, a, b)
y
## Calcule et évalue le graphe de différentiation automatique de y par rapport à x 
y.backward()
## Renvoie dy/dx
x.grad, a.grad, b.grad

::: panel-tabset

Le code qui suit correspond à l'application des informations disponibles dans la [documentation de torch](https://docs.pytorch.org/docs/stable/notes/extending.html#extending-torch-autograd) sur notre fonction example. 

On doit définir 4 méthodes: 
- `forward` qui reçoit les entrées et calcule la sortie
- `setup_context` qui stocke dans un objet `ctx` des tenseurs qui peuvent être réutilisés au moment du calcul de la dérivée
- `backward` (ou `vjp`) qui reçoit le gradient calculé en aval et renvoie le gradient, pour faire de la différentiation automatique en mode reverse. 
- `jvp` qui reçoit le gradient calculé en amont et renvoie le gradient, pour faire de la différentiation automatique en mode forward. 

## Définition de la fonction 


In [ ]:
class f_torch_manual(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(x, a, b):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache tensors for
        use in the backward pass using the ``ctx.save_for_backward`` method. Other
        objects can be stored directly as attributes on the ctx object, such as
        ``ctx.my_object = my_object``. Check out `Extending torch.autograd <https://docs.pytorch.org/docs/stable/notes/extending.html#extending-torch-autograd>`_
        for further details.
        """
        output = torch.tanh(torch.dot(a, x) + b)
        return output

    @staticmethod
    def setup_context(ctx, inputs, output):
        x, a, b = inputs
        ctx.result = output
        ## save output to cut computation time
        ctx.save_for_backward(x, a, b)

    @staticmethod
    def backward(ctx, grad_output):
        """        
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        It corresponds to a Vector Jacobian Product (vjp), used for reverse auto-differentiation
        """
        x, a, b = ctx.saved_tensors
        result  = ctx.result
        mult_fact = (1 - result.pow(2))
        grad_x = grad_output * a * mult_fact
        grad_a = grad_output * x * mult_fact
        grad_b = grad_output * mult_fact
        return grad_x, grad_a, grad_b # on doit calculer les grad par rapport à tous les arguments rajouter grad par rapport à a et b

    @staticmethod
    def jvp(x, a, b, tangents):
        """                
        It corresponds to a Jacobian Vector Product (jvp), used for forward auto-differentiation
        """
        ## Vector v of small perturbations
        tx, ta, tb = tangents
        ## Matrix (in this case vector) of first order gradient
        result = torch.tanh(torch.dot(a, x) + b)
        mult_fact = (1 - result.pow(2))
        Jx = a * mult_fact
        Ja = x * mult_fact
        Jb = mult_fact
        ## Return J(x, a, b)v
        return torch.dot(Jx, tx) + torch.dot(Ja, ta) + Jb * tb

## Dérivée par rapport à $x$, $a$ et $b$ 

### En mode reverse


In [ ]:
## Définit f
f = f_torch_manual.apply
z = f(x, a, b)
## Calcule et évalue le graphe de différentiation automatique de y par rapport à x 
## Réinitialise les gradients à zéro avant tout calcul 
x.grad.zero_(), a.grad.zero_(), b.grad.zero_()
z.backward()
## Renvoie dy/dx
x.grad, a.grad, b.grad

### En mode forward

On calcule la dérivée par rapport à la première coordonnée de $x$


In [ ]:
tangents = (torch.tensor([1, 0]).float(), torch.tensor([0, 0]).float(), torch.tensor(0).float())
f_torch_manual.jvp(x = x, a = a, b = b, tangents = tangents)

puis par rapport à la deuxième coordonnée de $a$


In [ ]:
tangents = (torch.tensor([0, 0]).float(), torch.tensor([0, 1]).float(), torch.tensor(0).float())
f_torch_manual.jvp(x = x, a = a, b = b, tangents = tangents)

Et on valide que les résultats obtenus coïncident avec ceux obtenus en mode reverse et directement en utilisant `torch` :grin:

:::

# Exemple en JAX [HG, AL]

Pour illustrer la différentiation automatique, nous allons utiliser JAX, sur une function simple dont nous connaissons les gradients analytiques.

## Dérivée par rapport à $x$


In [ ]:
import jax
import jax.numpy as jnp
from jax import grad, jit
from jax import random

## On définit une dimension arbitraire pour nos inputs
dim = 100

## On initialise les paramètres et le vecteur d'input de la fonction
a = jnp.arange(dim)/ (dim*10)
b = 0.5
x = (jnp.arange(dim) - 37) / (dim*10)

## On définit une fonction simple dont on connaît les gradients analytiques
def f(x, a, b):
    return jnp.tanh(jnp.dot(a, x) + b)

## On affiche la valeur de la fonction pour vérifier que tout est ok
f(x, a, b) 

::: panel-tabset

## Dérivée par rapport à $x$

Dans un premier temps, on peut définir les gradients exacts de cette fonction à partir d'une formule analytique. 


In [ ]:
def df_dx(x, a, b):
    return a * (1 - jnp.tanh(jnp.dot(a, x) + b) **2)

Puis on définit les gradients via autograd et on vérifie que les résultats sont identiques.


In [ ]:
## jax.grad calcule la formule backward par défaut
grad_df_dx = jax.grad(lambda x: f(x, a, b), argnums=0) 
## On peut aussi calculer la formule forward via jax.jacfwd
fwdgrad_df_dx = jax.jacfwd(lambda x: f(x, a, b), argnums=0) 

## On vérifie que les gradients retournent des valeurs identiques
assert jnp.allclose(grad_df_dx(x), df_dx(x, a, b))
assert jnp.allclose(grad_df_dx(x), fwdgrad_df_dx(x))

## Si les assertions ne retournent pas d'erreur, les gradients sont corrects
print('All good, we are ready to go!')

## Dérivée par rapport à $a$ et $b$

On définit également les gradients exacts par rapport aux paramètres $a$ et $b$ pour vérifier que l'on pourrait les optimiser dans un algorithme d'apprentissage.


In [ ]:
def df_dab(x, a, b):
    return x * (1 - jnp.tanh(jnp.dot(a, x) + b) **2), 1 - jnp.tanh(jnp.dot(a, x) + b) **2

Puis on définit ces gradients via autograd et on vérifie que les résultats sont identiques.


In [ ]:
## jax.grad calcule la formule backward par défaut
grad_df_dab = jax.grad(lambda a_b: f(x, *a_b), argnums=0)
## On peut aussi calculer la formule forward via jax.jacfwd
fwdgrad_df_dab = jax.jacfwd(lambda a_b: f(x, *a_b), argnums=0)

## On vérifie que les gradients retournent des valeurs identiques
assert all(jnp.allclose(grad_df_dab((a,b))[i], df_dab(x, a, b)[i]) for i in range(2))
assert all(jnp.allclose(grad_df_dab((a,b))[i], fwdgrad_df_dab((a,b))[i]) for i in range(2))

## Si les assertions ne retournent pas d'erreur, les gradients sont corrects
print('All good, we are ready to go!')

::: 

Nous avons donc bien vérifié que les gradients calculés avec JAX sont identiques aux gradients analytiques.

# Comparaison des temps de calcul en JAX entre autograd (backward et forward) et le calcul explicite des gradients

Pour mettre en lumière les différences entre l'autograd backward et forward, nous allons définir une nouvelle fonction $g$ dont les sorties sont de plus grandes dimensions que les entrées. 

::: panel-tabset

## $x \in \mathbb{R}^d$, f(x) $\in \mathbb{R}$

Comparaison des temps de calcul :

In [ ]:
## On lance une quantification du temps de calcul pour les trois méthodes (la commande .block_until_ready() est spécifique à JAX pour forcer l'évaluation des opérations asynchrones)
%timeit df_dx(x, a, b).block_until_ready()
%timeit grad_df_dx(x).block_until_ready()
%timeit fwdgrad_df_dx(x).block_until_ready()

## $x \in \mathbb{R}$, g(x) $\in \mathbb{R}^d$

On définit une nouvelle fonction $g$ qui retourne un vecteur de dimension $d$:


In [ ]:
## On définit la dimension du vecteur de sortie
dim_g = 100
## On initialise un point d'évaluation pour la fonction $g$
x_g = 0.5

## On définit la nouvelle fonction $g$ qui retourne un vecteur de dimension $dim_g$
def g(x):
    return jnp.array([jnp.tanh(x + i/dim_g) for i in range(dim_g)])

## On définit le gradient analytique de la fonction $g$
def dg_dx(x):
    return jnp.array([1- jnp.tanh(x + i/dim_g)**2 for i in range(dim_g)])
    
## jax.grad calcule la formule backward par défaut
grad_dg_dx = jax.jacrev(lambda x: g(x), argnums=0)
## On peut aussi calculer la formule forward via $jax.jacfwd$
fwdgrad_dg_dx = jax.jacfwd(lambda x: g(x), argnums=0)    

Comparaison des temps de calcul :

In [ ]:
%timeit dg_dx(x_g).block_until_ready()
%timeit grad_dg_dx(x_g).block_until_ready()
%timeit fwdgrad_dg_dx(x_g).block_until_ready()

:::

On onbserve que le calcul explicite des gradients est plus rapide que les deux autres méthodes. Il y a donc un prix à payer pour la différentiation automatique, bien qu'elle soit très utile pour des fonctions complexes. Cependant, nous allons voir qu'en combinaison avec une autre stratégie clef en JAX, le jitting, les choses changent.

# JIT compilation avec JAX 

La compilation JIT (Just-In-Time) permet d'optimiser les performances des fonctions en les compilant à la volée. JAX fournit la fonction `jit` pour cela.

::: panel-tabset

La syntaxe JAX est très confortable, il suffit d'ajouter le décorateur `@jit` avant la définition de la fonction. De manière analogue, on peut utiliser `jax.jit` à la suite de la fonction `jax.grad` ou `jax.jacfwd` pour compiler les gradients.

## Gradients jittés pour la fonction $f$

In [ ]:
## On redéfinit la fonction $f$ et ses gradients avec JIT
@jit
def f_jit(x, a, b):
    return jnp.tanh(jnp.dot(a, x) + b)

@jit
def df_dx_jit(x, a, b):
    return a * (1 - jnp.tanh(jnp.dot(a, x) + b) **2)

## On compile les gradients de la fonction $f$ avec JIT
grad_f_jit = jax.jit(jax.grad(lambda x: f_jit(x, a, b), argnums=0))
fwdgrad_f_jit = jax.jit(jax.jacfwd(lambda x: f_jit(x, a, b), argnums=0))

## On peut maintenant mesurer le temps de calcul des gradients avec JIT
%timeit df_dx_jit(x, a, b).block_until_ready()
%timeit grad_f_jit(x).block_until_ready()
%timeit fwdgrad_f_jit(x).block_until_ready()

## Gradients jittés pour la fonction $g$


In [ ]:
## On redéfinit la dimension du vecteur de sortie pour la fonction $g$ (les gradients sont sensibles à la dimension du vecteur de sortie, même avec JIT, notamment lorsque l'on compare l'autograd backward et forward)
dim_g = 10

## On redéfinit la fonction $f$ et ses gradients avec JIT
@jit
def g_jit(x):
    return jnp.array([jnp.tanh(x + i/dim_g) for i in range(dim_g)])
    
@jit
def dg_dx_jit(x):
    return jnp.array([1- jnp.tanh(x + i/dim_g)**2 for i in range(dim_g)])

## Pour que le calcul soit rapide aussi, il faut ajouter un 'vmap' avant le 'jit' pour que la fonction soit vectorisée et puisse tirer parti de la compilation JIT.
grad_g_jit = jax.jit(jax.vmap(jax.jacrev(lambda x: g_jit(x), argnums=0)))
fwdgrad_g_jit = jax.jit(jax.vmap(jax.jacfwd(lambda x: g_jit(x), argnums=0)))

## On peut maintenant mesurer le temps de calcul des gradients avec JIT
%timeit dg_dx_jit(x).block_until_ready()
%timeit grad_g_jit(x).block_until_ready()
%timeit fwdgrad_g_jit(x).block_until_ready()

:::

On peut constater dans les deux cas que la compilation JIT permet d'accélérer considérablement le calcul des gradients, et rendre l'autograd négligeable d'un point de vue computationel. Dans le cas de la fonction $g$ qui retourne un vecteur, la vectorisation permet aussi d'accélérer le calcul. Ainsi la combinaison de l'autograd, du jitting et de la vectorisation permet d'obtenir des performances optimales et rendre la différentiation automatique aussi efficace que l'analytique, même pour des fonctions complexes.

<!--
### Dérivée par rapport à $a$ et $b$
#
#::: 
#
#
### JVP et VJP
#
#Save below
#```{python}
#import jax
#import jax.numpy as jnp
#from jax import grad, jit, vmap
#from jax import random
#
#dim = 100
#
#a = jnp.arange(dim)/ (dim*10)
#b = 0.5
#
#def f(x, a, b):
#    return jnp.tanh(jnp.dot(a, x) + b)
#
#x = (jnp.arange(dim) - 37) / (dim*10)
#f(x, a, b) 
#
### jax.grad calcule la formule backward par défaut
#grad_df_dx = jax.grad(lambda x: f(x, a, b), argnums=0) 
#grad_df_dab = jax.grad(lambda a_b: f(x, *a_b), argnums=0)
#
#def df_dx(x, a, b):
#    return a * (1 - jnp.tanh(jnp.dot(a, x) + b) **2)
#def df_dab(x, a, b):
#    return x * (1 - jnp.tanh(jnp.dot(a, x) + b) **2), 1 - jnp.tanh(jnp.dot(a, x) + b) **2
#
#assert jnp.allclose(grad_df_dx(x), df_dx(x, a, b))
#
#
## assert jnp.allclose(jnp.array(grad_df_dab((a,b))), jnp.array(df_dab(x, a, b)))
#assert all(jnp.allclose(grad_df_dab((a,b))[i], df_dab(x, a, b)[i]) for i in range(2))
-->
::: panel-tabset

### JVP 

Retrouvons les mêmes résultats en redéfinissant les JVP [JAX doc](https://docs.jax.dev/en/latest/notebooks/Custom_derivative_rules_for_Python_code.html#custom-jvps-with-jax-custom-jvp)
Si un custom JVP est défini pour une fonction, il bypass tout les sous JVP qui pourraient y apparaître. Cette définion

JVP est capable de dévoiler une colonne de la jacobienne à la fois. Ce n'est pas adapté pour cette fonction. Une passe JVP ne peut dévoiler qu'une seule dérivée. Si l'on veut la dérivée par rapport à tout $x$ il nous faut faire $dim$ fois des JVP.


In [ ]:
@jax.custom_jvp
def f(x, a, b):
    return jnp.tanh(jnp.dot(a, x) + b)

@f.defjvp
def f_jvp(primals, tangents):
    x, a, b = primals
    x_dot, a_dot, b_dot = tangents
    primal_out = f(x, a, b)
    return primal_out, (jnp.dot(df_dx(x, a, b), x_dot) +  jnp.dot((x * (1 - jnp.tanh(jnp.dot(a, x) + b) **2)), a_dot) + jnp.dot((1 - jnp.tanh(jnp.dot(a, x) + b) **2), b_dot))

x0_tangents = jnp.zeros(dim)
x0_tangents = x0_tangents.at[0].set(1)
_, x_dot = jax.jvp(f, (x, a, b), (x0_tangents, jnp.zeros(dim), 0.))

assert jnp.allclose(grad_df_dx(x)[0], x_dot)

print("All right!")

### VJP

[jax doc](https://docs.jax.dev/en/latest/notebooks/Custom_derivative_rules_for_Python_code.html#use-jax-custom-vjp-to-define-custom-reverse-mode-only-rules)


In [ ]:
@jax.custom_vjp
def f(x, a, b):
    return jnp.tanh(jnp.dot(a, x) + b)

def f_fwd(x, a, b):
    return f(x, a, b), (a * (1 - jnp.tanh(jnp.dot(a, x) + b) **2), x * (1 - jnp.tanh(jnp.dot(a, x) + b) **2), 1 - jnp.tanh(jnp.dot(a, x) + b) **2)

def f_bwd(res, g):
    x_, a_, b_ = res
    return (x_ * g,  a_ * g, b_ * g)
    #return (jnp.dot(x_, g),  jnp.dot(a_, g), jnp.dot(b_, g))

f.defvjp(f_fwd, f_bwd)
_, f_vjp = jax.vjp(f, x, a, b)

assert jnp.allclose(grad_df_dx(x), f_vjp(1.)[0])
assert all(jnp.allclose(grad_df_dab((a,b))[i], f_vjp(1.)[i + 1]) for i in range(2))
print("All right!")

Note that redefining the VJP redefines the grad function (which passes by f_fwd), and note the equivalency:


In [ ]:
assert jnp.allclose(f_vjp(1.)[0], jax.grad(f)(x, a, b))
print("All right!")

:::

We conclude that it is much more natural to use the forward mode for this function because...